LOAD DATA

In [2]:
import os
import pandas as pd
from twkorean import TwitterKoreanProcessor
from tqdm import tqdm
from gensim.models import Word2Vec
from review_scorer import ReviewScorer
import json

ModuleNotFoundError: No module named 'review_scorer'

In [4]:
HOME = os.getcwd().split(os.path.sep)
PROJ_FOLDER = HOME + ['PycharmProjects', 'knlp_test']
DATA_FOLDER = PROJ_FOLDER + ['data']

In [5]:
senti: pd.DataFrame
data: pd.DataFrame

with open(os.path.sep.join(DATA_FOLDER + ['SentiWord_info.json']),
          'rt',
          encoding='UTF8') as f:
    senti = pd.DataFrame.from_dict(json.load(f))

with open(os.path.sep.join(DATA_FOLDER + ['learning_total.json']),
          'rt',
          encoding='UTF8') as f:
    data = pd.DataFrame.from_dict(json.load(f))

PREPROCESS

In [6]:
senti['polarity'] = senti['polarity'].apply(lambda x: int(x))

In [7]:
data.drop('index', axis=1, inplace=True)
data.columns = ['review', 'overall', 'taste', 'service', 'price', 'atmosphere', 'name', 'location']
data.loc[:,['overall', 'taste', 'service', 'price', 'atmosphere']] = data.loc[:,['overall', 'taste', 'service', 'price', 'atmosphere']].apply(lambda x: x//10 if x is not -1 else x)
data.head()

,review,overall,taste,service,price,atmosphere,name,location
0,음 서양식 입니다. 외국에서 자주 먹어서 그런지 몰라도 그냥 그냥. 1/1 점심에 ...,3,3,3,2,-1,라이너스 바베큐,서울 용산구 이태원로 136-13 Down the stairway next to M...
1,"늘 사람 많은 바베큐맛집 라이너스바베큐, 그래도 대기 시스템이 도입되어서 전보다 편...",5,-1,-1,-1,-1,라이너스 바베큐,서울 용산구 이태원로 136-13 Down the stairway next to M...
2,언제나 방문할때마다 맛있게 먹는 라이너스~! 강추해요. 용산은 라이너스 때문에 가는...,5,-1,-1,-1,-1,라이너스 바베큐,서울 용산구 이태원로 136-13 Down the stairway next to M...
3,라이너스 바비큐 맛있어요 이태원와서 맛있게 먹었네요 ㅎㅎㅎ 오랜만에 서울와서 좋은...,5,-1,-1,-1,-1,라이너스 바베큐,서울 용산구 이태원로 136-13 Down the stairway next to M...
4,여긴 정말 맛잇는 곳이죠. 한국에 사는 외국인 친구가 소개해서 가게되었습니다. 마치...,5,-1,-1,-1,-1,라이너스 바베큐,서울 용산구 이태원로 136-13 Down the stairway next to M...


TOKENIZE

In [8]:
processor = TwitterKoreanProcessor()
reviews = [processor.tokenize_to_strings(row.review) for _, row in tqdm(data.iterrows())]

79812it [04:22, 303.79it/s]


TRAIN WORD2VEC

In [13]:
model = Word2Vec(sentences=reviews, vector_size=100, window=5, min_count=2, workers=4, hs=1)

In [18]:
model.wv.most_similar(positive=['존맛'], topn=20)

[('고아원', 0.510647177696228),
 ('place', 0.49208980798721313),
 ('was', 0.4724150598049164),
 ('젓갈', 0.42392709851264954),
 ('심장', 0.42013952136039734),
 ('perfect', 0.41733941435813904),
 ('The', 0.41377753019332886),
 ('SSOo', 0.4108841121196747),
 ('다스', 0.4071258306503296),
 ('Nice', 0.4050728976726532),
 ('돌려주다', 0.39864009618759155),
 ('아동', 0.39749589562416077),
 ('산입', 0.3957109749317169),
 ('̆̈', 0.3947729766368866),
 ('부레', 0.3911246657371521),
 ('에여', 0.38914161920547485),
 ('환경호르몬', 0.38837218284606934),
 ('/'', 0.3850107192993164),
 ('포디', 0.3828795254230499),
 ('도저히', 0.3828708827495575)]

TAG SENTIMENTAL DICTIONARY

In [9]:
from typing import List

def get_similar_words_indexes(words: List[str],
                          senti_df: pd.DataFrame,
                          model: Word2Vec):
    words_set = set(w[0] for word in words for w in model.wv.most_similar(word, topn=100))
    words_set.update(words)
    return senti_df['word'].isin(words_set) | senti_df['word_root'].isin(words_set)

In [10]:
senti.loc[get_similar_words_indexes(words=['맛있다', '맛없다'],
                      senti_df=senti, model=model)]

,word,word_root,polarity
80,가능하다,가능,2
295,감격,감격,2
297,감격스럽다,감격,2
298,감격스레,감격,2
299,감격이,감격,2
301,감격하다,감격,2
302,감격하여,감격,2
416,강하다,강하,1
1004,고요하게,고요,1
1006,고요하고,고요,1


In [11]:
def tag_senti(taste_words: List[str],
              price_words: List[str],
              atmosphere_words: List[str],
              service_words: List[str],
              senti_df: pd.DataFrame,
              model: Word2Vec):
    categories = {"taste": taste_words,
                  "price": price_words,
                  "atmosphere": atmosphere_words,
                  "service": service_words}

    for category, words in categories.items():
        senti_df[category] = False
        senti_df.loc[get_similar_words_indexes(words=words,
                                               senti_df=senti_df,
                                               model=model), category] = True

In [12]:
tag_senti(taste_words=['맛', '맛있다', '맛없다', '부드럽다'],
          price_words=['가격', '비싸다', '싸다', '저렴'],
          atmosphere_words=['인테리어', '분위기'],
          service_words=['친절', '친절하다', '서비스'],
          senti_df=senti,
          model=model)

In [13]:
senti.head()

,word,word_root,polarity,taste,price,atmosphere,service
0,(-;,(,1,False,False,False,False
1,(;_;),(;_;),-1,False,False,False,False
2,(^^),(^^),1,False,False,False,False
3,(^-^),(^-^),1,False,False,False,False
4,(^^*,(,1,False,False,False,False


In [18]:
def score_review(review: str, senti: pd.DataFrame)->dict:
    scores = {"taste": 0,
              "price": 0,
              "atmosphere": 0,
              "service": 0}
    tokens = processor.tokenize_to_strings(review)
    scoring_review = senti.loc[senti.word.isin(tokens)]

    return {category: sum(scoring_review.loc[scoring_review.loc[:,category] == True, 'polarity']) for category, score in scores.items()}

In [15]:
score_review(data.review[0], senti)

{'taste': 2, 'price': 2, 'atmosphere': 0, 'service': 2}

In [16]:
for i in tqdm(range(0, 20)):
    print(score_review(data.iloc[i]['review'], senti))
    print(data.iloc[i]['review'])

 10%|████████▍                                                                           | 2/20 [00:00<00:01, 15.22it/s]

{'taste': 2, 'price': 2, 'atmosphere': 0, 'service': 2}
음 서양식 입니다. 외국에서 자주 먹어서 그런지 몰라도 그냥 그냥. 1/1 점심에 갔는데 사람이 많네요. 줄 서서 먹을 정도는 아닌데. 아이디어만 좋음. 테이블 안내 하는 직원 서비스도는 별루..
{'taste': 4, 'price': 4, 'atmosphere': 0, 'service': 2}
늘 사람 많은 바베큐맛집 라이너스바베큐, 그래도 대기 시스템이 도입되어서 전보다 편한 것 같아요. 여기서 먹는 바베큐 참 맛있어요 추천
{'taste': 2, 'price': 2, 'atmosphere': 0, 'service': 0}
언제나 방문할때마다 맛있게 먹는 라이너스~! 강추해요. 용산은 라이너스 때문에 가는거같네요 ㅎㅎ 패티번은 언제나 추가를 해야 됩니다


 35%|█████████████████████████████▍                                                      | 7/20 [00:00<00:00, 18.58it/s]

{'taste': 4, 'price': 4, 'atmosphere': 0, 'service': 2}
라이너스 바비큐 맛있어요 이태원와서 맛있게 먹었네요 ㅎㅎㅎ  오랜만에 서울와서 좋은 시간 보내고 가요 ㅎㅎㅎ굿굿굿굿
{'taste': 6, 'price': 2, 'atmosphere': 0, 'service': 2}
여긴 정말 맛잇는 곳이죠. 한국에 사는 외국인 친구가 소개해서 가게되었습니다. 마치 우리가 외국에서 맛있는 한식을 먹는 그런 느낌으로 외국인 친구가 참 좋아하더군요. 우리나라 입맛에도 훌륭합니다
{'taste': 2, 'price': 2, 'atmosphere': 0, 'service': 0}
가격이 저렴하지도 않고, 그렇다고 맛이 특별하지도 않은데 끝없이 늘어선 줄. 명성에 이끌려 방문했으나 다시는 방문하고 싶은 생각이 들지 않는 평범한 바베큐전문점.
{'taste': 0, 'price': 0, 'atmosphere': 2, 'service': 0}
그저 그런 평범한 음식, 그럼에도 불구하고 긴 대기시간, 굉장히 불친절한 서비스. 굳이 애써 기다릴 만한 가치가 없는 곳입니다.
{'taste': 3, 'price': 3, 'atmosphere': 0, 'service': 2}
날씨좋을때 먹는 분위기도 좋고 음식도 좋아요 저녁 이른 시간에 맞춰가면 괜찮지만 조금만 지체되어도 대기를 제법해야합니다. 오랜 시간을 대기해서 먹기는 꺼려져요
{'taste': 2, 'price': 2, 'atmosphere': 0, 'service': 0}
라이너스 바베큐 이태우ㅏㄴ에  찾아가기가 꼬불꼬불 복잡하지만 잫 찾아갔어요 맛있구요  배부르게 잘 먹었어용 ㅎㅎㅎㅎ


 75%|██████████████████████████████████████████████████████████████▎                    | 15/20 [00:00<00:00, 27.77it/s]

{'taste': 1, 'price': 0, 'atmosphere': 0, 'service': 0}
플레이트가 재미있어서 두번정도 방문했습니다. 그런데 저에게는 확실히 조금은 질리는 맛이기는 해요. 한번도 안드셔보셨다면 한번정도 방문하시는 것을 추천드립니다.
{'taste': 2, 'price': 2, 'atmosphere': 0, 'service': 0}
1.가격이 비쌈 2.먹기 불편함 (인원이 많다면 더더욱) 3.그래도 고기와 빵 야채는 맛있음 4.사이드로 선택이 많아서 좋음 5.물티슈는 필수...
{'taste': 2, 'price': 2, 'atmosphere': 0, 'service': 2}
친구 3명이랑 야외에서 먹어요~ 번에 고기 넣어먹으면 꿀맛이지만 웨이팅이 길때도 있어서 집가까운 친구분이 먼저 가계시면 좋습니다!
{'taste': 4, 'price': 4, 'atmosphere': 0, 'service': 2}
4명이서 식사를 했습니다. 4인용 플래터를 시키고 맥주를 한잔씩 했습니다. 맛이 있었으나 고기가 차가웠어요. 인테리어, 야외 분위기 좋습니다. 유명 맛집이라지만 이정도 맛을 내는 바비큐집은 다른데도 많은 것 같습니다.
{'taste': 2, 'price': 2, 'atmosphere': 0, 'service': 2}
피크닉테이블에서 플래터 주문해서 먹었는데 립이 엄청 커서 함께 간 사람들 모두 대만족이었구요 ㅋㅋ 사이드로 나온 빵에 고기를 끼워먹으니 이보다 더 맛있을 수 없엇어요 ㅠㅠ 소스도 취향껏 찍어먹으면 되서 좋구요, 가격이 조금 비싸지만 강추입니다 ㅎ
{'taste': 4, 'price': 4, 'atmosphere': 0, 'service': 2}
고기와 빵, 프렌치프라이, 샐러드 한플레이트에 넉넉한 양으로 푸짐하게 먹을 수 있어서 너무 좋았습니다. 2인 플레이트에 사이드메뉴 2가지 추가해서 먹었는데 너무 맛있었고 달빛필스너와 맥주 한가지를 같이 주문해서 먹었는데 너무 좋았습니다. 지하에 이렇게 분위기 좋은 바베큐집이 있는줄 몰

100%|███████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 23.20it/s]

{'taste': -1, 'price': 0, 'atmosphere': 0, 'service': 0}
큰 기대를 하고 가서 그런지 생각보다 맛이 그저 그랬어요. 사실 맛없으면 안되는 조합인데, 그냥 무난한 정도더라구요.
{'taste': 3, 'price': 4, 'atmosphere': 0, 'service': 2}
햄버거 하나 먹자고 이리 오래 기다려보기는 처음이네요. 그만큼 사람이 많고 인기가 대단하네요.. 분위기 좋고 맛도 괜찮지만 가격은 괜찮지 않아요..
{'taste': 4, 'price': 3, 'atmosphere': 0, 'service': 2}
한시간은 기다렸던 것 같다. 그래도 요즘 세상 좋아져서, 아이패드 같은 곳에 연락처 적고 웨이팅 리스트 올리면, 들어갈때 되어서 폰으로 연락이 온다. 분위기도 적당히 좋고, 맛도 양도 적당히 좋다. 그렇지만 다음에 갔을때 또 한시간이나 기다리긴 싫다.
{'taste': 3, 'price': 1, 'atmosphere': 0, 'service': 0}
대기해서 먹으면 실망할 맛이지만 그냥 괜찮아요. 다양한 종류를 먹는 재미가 있습니다. 평일 점심에 많이 안 기다릴 때 가세요


In [34]:
def compare_score(index:int, _data: pd.DataFrame, _senti: pd.DataFrame):
    print('computed ->\t', *score_review(_data.iloc[index]['review'], senti).values())
    print('real -> \t', *_data.iloc[index, 2:6])
    print(_data.iloc[i]['review'])

In [106]:
bbq = data.loc[data['name'] == '바토스 이태원점'].copy()

In [107]:
# ['overall','taste','service','price','atmosphere']
bbq.loc[:, 'overall'] = bbq.loc[:, 'overall'].apply(lambda x: 0 if x == -1 else x)
bbq.loc[:, 'taste'] = bbq.loc[:, 'taste'].apply(lambda x: 0 if x == -1 else x)
bbq.loc[:, 'service'] = bbq.loc[:, 'service'].apply(lambda x: 0 if x == -1 else x)
bbq.loc[:, 'price'] = bbq.loc[:, 'price'].apply(lambda x: 0 if x == -1 else x)
bbq.loc[:, 'atmosphere'] = bbq.loc[:, 'atmosphere'].apply(lambda x: 0 if x == -1 else x)

In [108]:
def get_avg(col: pd.Series):
    cnt = 0
    sum = 0
    for i in col:
        if i != 0:
            cnt += 1
            sum += i
    return sum / cnt

In [109]:
print(get_avg(bbq.overall))
print(get_avg(bbq.taste))
print(get_avg(bbq.service))
print(get_avg(bbq.price))
print(get_avg(bbq.atmosphere))

4.149862930246726
4.444711538461538
4.055288461538462
3.5376602564102564
5.0


In [110]:
bbq_scores = pd.DataFrame([score_review(item.review, senti).values() for _, item in tqdm(bbq.iterrows())])
bbq_scores.columns= ['taste', 'price', 'atmosphere', 'service']

3283it [02:35, 21.10it/s]


In [111]:
import numpy as np

In [112]:
overall = []
for i, row in bbq_scores.iterrows():
    overall.append(np.average(row.loc[['taste', 'price', 'atmosphere', 'service']]))
bbq_scores['overall'] = overall

In [113]:
print(get_avg(bbq_scores.overall))
print(get_avg(bbq_scores.taste))
print(get_avg(bbq_scores.service))
print(get_avg(bbq_scores.price))
print(get_avg(bbq_scores.atmosphere))

1.6080213903743314
3.157510564733
2.096774193548387
2.226958993097848
2.0
